# Import necessary libraries

In [ ]:
import torch
import torch.nn as nn

from collections import Counter

In [ ]:
# documents(dataset)
docs = '''
i want to buy a computer
she does not love me
they want to go there
he loves her laptop
you do not love me
i want to get a good mark
'''

In [ ]:
words = [word for word in docs.split()]
list(sorted(Counter(words)))

['a',
 'buy',
 'computer',
 'do',
 'does',
 'get',
 'go',
 'good',
 'he',
 'her',
 'i',
 'laptop',
 'love',
 'loves',
 'mark',
 'me',
 'not',
 'she',
 'there',
 'they',
 'to',
 'want',
 'you']

# Create a tokenizer

In [ ]:
class Tokenizer:
  def __init__(self):
    super().__init__()
    self.word2idx = {}
    self.idx2word = {}
    self.word_count = 1

    self.word2idx["<PAD>"] = 0
    self.idx2word[0] = "<PAD>"

  def __len__(self):
    return len(self.word2idx)

  def fit(self, docs):
    words = []
    for word in docs.split():
      words.append(word)
    list_words = list(sorted(Counter((words))))

    for index, word in enumerate(list_words, start=1):
      self.word2idx[word] = index
      self.idx2word[index] = word
      self.word_count += 1
  def encode(self, text):
    text = text.lower()
    tokenized_words = []

    for word in text.split():
      if word in self.word2idx:
        tokenized_words.append(self.word2idx[word])
      else:
        tokenized_words.append(self.word2idx["<PAD>"])

    return tokenized_words

  def decode(self, idxs):
    decoded_text = []

    for idx in idxs:
      decoded_text.append(self.idx2word[idx])

    return decoded_text

In [ ]:
tokenizer = Tokenizer()

In [ ]:
tokenizer.fit(docs)

In [ ]:
inputs = tokenizer.encode("i want to buy a car")
inputs

[11, 22, 21, 2, 1, 0]

In [ ]:
tokenizer.decode(inputs)

['i', 'want', 'to', 'buy', 'a', '<PAD>']

# Prepare dataset

In [ ]:
input_sequences = []
output_sequence = []

In [ ]:
for line in docs.strip().split('\n'):
  token_line = tokenizer.encode(line)

  for i in range(1, len(token_line)):
    n_gram = token_line[:i+1]

    input_sequences.append(n_gram[:-1])
    output_sequence.append(n_gram[-1])

In [ ]:
max_len_sequence = max([len(seq) for seq in input_sequences])
max_len_sequence

6

In [ ]:
padded_sequences = []

In [ ]:
for seq in input_sequences:
  len_seq = len(seq)
  pad_with = max_len_sequence - len_seq

  padded_sequences.append([0] * pad_with + seq)

In [ ]:
padded_sequences

[[0, 0, 0, 0, 0, 11],
 [0, 0, 0, 0, 11, 22],
 [0, 0, 0, 11, 22, 21],
 [0, 0, 11, 22, 21, 2],
 [0, 11, 22, 21, 2, 1],
 [0, 0, 0, 0, 0, 18],
 [0, 0, 0, 0, 18, 5],
 [0, 0, 0, 18, 5, 17],
 [0, 0, 18, 5, 17, 13],
 [0, 0, 0, 0, 0, 20],
 [0, 0, 0, 0, 20, 22],
 [0, 0, 0, 20, 22, 21],
 [0, 0, 20, 22, 21, 7],
 [0, 0, 0, 0, 0, 9],
 [0, 0, 0, 0, 9, 14],
 [0, 0, 0, 9, 14, 10],
 [0, 0, 0, 0, 0, 23],
 [0, 0, 0, 0, 23, 4],
 [0, 0, 0, 23, 4, 17],
 [0, 0, 23, 4, 17, 13],
 [0, 0, 0, 0, 0, 11],
 [0, 0, 0, 0, 11, 22],
 [0, 0, 0, 11, 22, 21],
 [0, 0, 11, 22, 21, 6],
 [0, 11, 22, 21, 6, 1],
 [11, 22, 21, 6, 1, 8]]

In [ ]:
X = torch.tensor(padded_sequences, dtype=torch.long)
y = torch.tensor(output_sequence, dtype=torch.long)

In [ ]:
X, y

(tensor([[ 0,  0,  0,  0,  0, 11],
         [ 0,  0,  0,  0, 11, 22],
         [ 0,  0,  0, 11, 22, 21],
         [ 0,  0, 11, 22, 21,  2],
         [ 0, 11, 22, 21,  2,  1],
         [ 0,  0,  0,  0,  0, 18],
         [ 0,  0,  0,  0, 18,  5],
         [ 0,  0,  0, 18,  5, 17],
         [ 0,  0, 18,  5, 17, 13],
         [ 0,  0,  0,  0,  0, 20],
         [ 0,  0,  0,  0, 20, 22],
         [ 0,  0,  0, 20, 22, 21],
         [ 0,  0, 20, 22, 21,  7],
         [ 0,  0,  0,  0,  0,  9],
         [ 0,  0,  0,  0,  9, 14],
         [ 0,  0,  0,  9, 14, 10],
         [ 0,  0,  0,  0,  0, 23],
         [ 0,  0,  0,  0, 23,  4],
         [ 0,  0,  0, 23,  4, 17],
         [ 0,  0, 23,  4, 17, 13],
         [ 0,  0,  0,  0,  0, 11],
         [ 0,  0,  0,  0, 11, 22],
         [ 0,  0,  0, 11, 22, 21],
         [ 0,  0, 11, 22, 21,  6],
         [ 0, 11, 22, 21,  6,  1],
         [11, 22, 21,  6,  1,  8]]),
 tensor([22, 21,  2,  1,  3,  5, 17, 13, 16, 22, 21,  7, 19, 14, 10, 12,  4, 17,
       

# Create a model

In [ ]:
class TextGenerator(nn.Module):
  def __init__(self, vocab_size, embedding_dim, hidden_dim):
    super().__init__()
    self.vocab_size = vocab_size
    self.embedding_dim = embedding_dim
    self.hidden_dim = hidden_dim

    self.embedding = nn.Embedding(num_embeddings=vocab_size, embedding_dim=embedding_dim)
    self.lstm = nn.LSTM(input_size=embedding_dim, hidden_size=hidden_dim, batch_first=True)
    self.out = nn.Linear(hidden_dim, vocab_size)

  def forward(self, x):
    embedded = self.embedding(x)
    lstm_output, (_, _) = self.lstm(embedded)
    x = lstm_output[:, -1, :]
    x = self.out(x)

    return x

# CONFIGURATION

In [ ]:
VOCAB_SIZE = len(tokenizer)
EMBEDDING_DIM = 64
HIDDEN_DIM = 32
EPOCHS = 200

In [ ]:
generator = TextGenerator(vocab_size=VOCAB_SIZE, embedding_dim=EMBEDDING_DIM, hidden_dim=HIDDEN_DIM)

# Train model

In [ ]:
for param in generator.parameters():
  print(param.size())

torch.Size([24, 64])
torch.Size([128, 64])
torch.Size([128, 32])
torch.Size([128])
torch.Size([128])
torch.Size([24, 32])
torch.Size([24])


In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(generator.parameters(), lr=0.001)

In [ ]:
loss = 0
total_correct = 0

In [ ]:
for epoch in range(EPOCHS):
  generator.train()

  optimizer.zero_grad()

  outputs = generator(X)

  loss_ = criterion(outputs, y)

  loss_.backward()
  optimizer.step()
  loss += loss_.item()

  if epoch % 10 == 0:
    total_correct = (outputs.argmax(dim=1) == y).sum().item()

    print(f"epoch: {epoch}, accuracy: {total_correct / (y.size(0))}, loss: {loss_:.3f}")


epoch: 0, accuracy: 0.9615384615384616, loss: 0.056
epoch: 10, accuracy: 0.9615384615384616, loss: 0.056
epoch: 20, accuracy: 0.9615384615384616, loss: 0.056
epoch: 30, accuracy: 0.9615384615384616, loss: 0.056
epoch: 40, accuracy: 0.9615384615384616, loss: 0.056
epoch: 50, accuracy: 0.9615384615384616, loss: 0.056
epoch: 60, accuracy: 0.9615384615384616, loss: 0.056
epoch: 70, accuracy: 0.9615384615384616, loss: 0.055
epoch: 80, accuracy: 0.9615384615384616, loss: 0.055
epoch: 90, accuracy: 0.9615384615384616, loss: 0.055
epoch: 100, accuracy: 0.9615384615384616, loss: 0.055
epoch: 110, accuracy: 0.9615384615384616, loss: 0.055
epoch: 120, accuracy: 0.9615384615384616, loss: 0.055
epoch: 130, accuracy: 0.9615384615384616, loss: 0.055
epoch: 140, accuracy: 0.9615384615384616, loss: 0.055
epoch: 150, accuracy: 0.9615384615384616, loss: 0.055
epoch: 160, accuracy: 0.9615384615384616, loss: 0.055
epoch: 170, accuracy: 0.9615384615384616, loss: 0.055
epoch: 180, accuracy: 0.961538461538461

# Generate text

In [ ]:
text = "i want to buy a"
text

'i want to buy a'

In [ ]:
tokenizer.encode("computer")

[3]

In [ ]:
inputs = tokenizer.encode(text)
inputs

[11, 22, 21, 2, 1]

In [ ]:
inputs_ = torch.tensor(inputs, dtype=torch.long)
inputs_

tensor([11, 22, 21,  2,  1])

In [ ]:
inputs_.unsqueeze(0)

tensor([[11, 22, 21,  2,  1]])

In [ ]:
outputs = generator(inputs_.unsqueeze(0))
outputs

tensor([[-0.4316,  1.5806,  0.1193,  8.5805, -1.0921, -1.0784, -0.4032, -0.5029,
          5.3135, -0.5916, -1.8023, -0.5370, -0.0980,  1.7519, -2.3480,  0.4038,
         -0.2747, -0.6938, -0.3964,  1.5759, -0.0267, -1.2294, -3.6248, -0.3102]],
       grad_fn=<AddmmBackward0>)

In [ ]:
next_token = outputs.argmax(dim=1)
next_token

tensor([3])

In [ ]:
tokenizer.decode([int(next_token)])

['computer']

In [ ]:
def generate_text(generator, text):
  token_list = tokenizer.encode(text)
  inputs = torch.tensor(token_list, dtype=torch.long)
  outputs = generator(inputs.unsqueeze(0))
  predicted_token = outputs.argmax(dim = 1)
  predicted_word = tokenizer.decode([int(predicted_token)])
  return text + " " + predicted_word[0]

In [ ]:
generate_text(generator, 'she does not')

'she does not love'

In [ ]:
def generate_words(generator, text, next_words):
  generator.eval()

  for _ in range(next_words):


# text generation 2

# import necessary libraries

In [ ]:
import torch
import torch.nn as nn
from torch.optim import Adam

import re

from collections import Counter

In [ ]:
corpus = [
    "I want to buy a new computer but I do not have money.",
    "She likes to read books when it rains outside.",
    "He goes to the gym every morning before work.",
    "I am learning how to cook Italian food.",
    "They are planning to visit Paris next summer.",
    "My phone battery dies very fast these days.",
    "I need to finish my homework before watching TV.",
    "The weather is getting colder every day.",
    "We are trying to save money for a vacation.",
    "He wants to learn how to play the guitar.",
    "I love listening to music when I feel sad.",
    "She is studying hard to pass her final exams.",
    "I forgot my keys at home this morning.",
    "They were talking about movies last night.",
    "My brother wants to buy a new bicycle.",
    "I am thinking about starting a YouTube channel.",
    "We walked in the park and watched the sunset.",
    "He drinks too much coffee during the day.",
    "She wants to learn another language next year.",
    "I saw a beautiful cat near my house yesterday.",
    "They are building a new shopping mall in the city.",
    "My friend is always late for our meetings.",
    "I hope to get a better job in the future.",
    "The internet connection is very slow today.",
    "I bought some fresh vegetables from the market.",
    "He forgot to call me back after the meeting.",
    "She spends too much time scrolling on her phone.",
    "We are watching a funny movie tonight.",
    "I am trying to eat healthier and exercise more.",
    "He is learning to code in Python for his new project.",
    "I woke up late and missed my morning class.",
    "They are working on a group project together.",
    "She wants to paint her room a different color.",
    "I am waiting for the bus in the cold.",
    "He enjoys taking photos of nature.",
    "We decided to stay home and order pizza.",
    "I need to clean my room before guests arrive.",
    "She forgot to bring her laptop to school.",
    "He is thinking about moving to another city.",
    "I love the sound of rain at night.",
    "They often play football on weekends.",
    "My parents are coming to visit next week.",
    "I want to start reading more this year.",
    "He doesn’t like waking up early.",
    "She always drinks tea in the morning.",
    "We are saving money to buy a car.",
    "I forgot to charge my phone last night.",
    "They are watching a documentary about space.",
    "He wants to open his own business someday.",
    "I need to buy some groceries after work.",
    "She is learning how to bake a cake.",
    "We went to the beach and took many photos.",
    "I am trying to spend less time online.",
    "He likes to listen to podcasts during his commute.",
    "She has to finish her report before tomorrow.",
    "They want to renovate their old house.",
    "I am planning to visit my grandparents soon.",
    "He often helps his younger brother with homework.",
    "She bought a new dress for the party.",
    "We love to travel and explore new places.",
    "I forgot to turn off the lights in my room.",
    "They are discussing a new idea for a startup.",
    "He always eats breakfast before leaving home.",
    "She is preparing for an important interview.",
    "We were talking about our favorite movies.",
    "I want to learn how to swim properly.",
    "He spends a lot of time playing video games.",
    "She likes watching the stars at night.",
    "We are learning how to dance together.",
    "I met my friend at the coffee shop.",
    "He is reading a book about psychology.",
    "They are waiting for the results of the test.",
    "She enjoys walking in the park every evening.",
    "I need to buy a new pair of shoes.",
    "We went shopping and bought some clothes.",
    "He is watching football with his friends.",
    "She likes cooking for her family on weekends.",
    "I cleaned my desk and organized my books.",
    "They want to adopt a puppy from the shelter.",
    "He was talking about his new job offer.",
    "She is trying to wake up earlier every day.",
    "We love spending time with our family.",
    "I want to improve my English skills.",
    "He forgot to take out the trash.",
    "She is looking for a new apartment.",
    "We decided to cook dinner together.",
    "I need to practice driving before my exam.",
    "He likes to draw in his free time.",
    "She studies biology at the university.",
    "They enjoy watching comedy shows on TV.",
    "I am waiting for my order to arrive.",
    "He is always kind to everyone he meets.",
    "She loves animals and wants to be a vet.",
    "We were laughing at a funny joke.",
    "I want to go hiking next weekend.",
    "He plays guitar in a local band.",
    "She is helping her mom in the kitchen.",
    "We are talking about our future plans.",
    "I like spending evenings reading a book.",
    "He wants to travel around the world someday.",
    "She forgot to send an important email.",
    "We are celebrating his birthday tonight."
]


In [ ]:
for line in corpus:
  line = re.sub(r"[^a-zA-Z ]", "", line)
  line = line + " <EOS>"

In [ ]:
corpus = [re.sub(r"[^a-zA-Z ]", "", line) + " <EOS>" for line in corpus]

In [ ]:
corpus

['I want to buy a new computer but I do not have money EOS <EOS>',
 'She likes to read books when it rains outside EOS <EOS>',
 'He goes to the gym every morning before work EOS <EOS>',
 'I am learning how to cook Italian food EOS <EOS>',
 'They are planning to visit Paris next summer EOS <EOS>',
 'My phone battery dies very fast these days EOS <EOS>',
 'I need to finish my homework before watching TV EOS <EOS>',
 'The weather is getting colder every day EOS <EOS>',
 'We are trying to save money for a vacation EOS <EOS>',
 'He wants to learn how to play the guitar EOS <EOS>',
 'I love listening to music when I feel sad EOS <EOS>',
 'She is studying hard to pass her final exams EOS <EOS>',
 'I forgot my keys at home this morning EOS <EOS>',
 'They were talking about movies last night EOS <EOS>',
 'My brother wants to buy a new bicycle EOS <EOS>',
 'I am thinking about starting a YouTube channel EOS <EOS>',
 'We walked in the park and watched the sunset EOS <EOS>',
 'He drinks too much c

In [ ]:
words = []

In [ ]:
for line in corpus:
  line = line.lower()
  line = re.sub(r"[^a-zA-Z ]", "", line)
  for line_ in line.split():
    words.append(line_)

In [ ]:
words

['i',
 'want',
 'to',
 'buy',
 'a',
 'new',
 'computer',
 'but',
 'i',
 'do',
 'not',
 'have',
 'money',
 'eos',
 'eos',
 'she',
 'likes',
 'to',
 'read',
 'books',
 'when',
 'it',
 'rains',
 'outside',
 'eos',
 'eos',
 'he',
 'goes',
 'to',
 'the',
 'gym',
 'every',
 'morning',
 'before',
 'work',
 'eos',
 'eos',
 'i',
 'am',
 'learning',
 'how',
 'to',
 'cook',
 'italian',
 'food',
 'eos',
 'eos',
 'they',
 'are',
 'planning',
 'to',
 'visit',
 'paris',
 'next',
 'summer',
 'eos',
 'eos',
 'my',
 'phone',
 'battery',
 'dies',
 'very',
 'fast',
 'these',
 'days',
 'eos',
 'eos',
 'i',
 'need',
 'to',
 'finish',
 'my',
 'homework',
 'before',
 'watching',
 'tv',
 'eos',
 'eos',
 'the',
 'weather',
 'is',
 'getting',
 'colder',
 'every',
 'day',
 'eos',
 'eos',
 'we',
 'are',
 'trying',
 'to',
 'save',
 'money',
 'for',
 'a',
 'vacation',
 'eos',
 'eos',
 'he',
 'wants',
 'to',
 'learn',
 'how',
 'to',
 'play',
 'the',
 'guitar',
 'eos',
 'eos',
 'i',
 'love',
 'listening',
 'to',
 'mus

In [ ]:
set_words = list(sorted(Counter(words)))
set_words

['a',
 'about',
 'adopt',
 'after',
 'always',
 'am',
 'an',
 'and',
 'animals',
 'another',
 'apartment',
 'are',
 'around',
 'arrive',
 'at',
 'back',
 'bake',
 'band',
 'battery',
 'be',
 'beach',
 'beautiful',
 'before',
 'better',
 'bicycle',
 'biology',
 'birthday',
 'book',
 'books',
 'bought',
 'breakfast',
 'bring',
 'brother',
 'building',
 'bus',
 'business',
 'but',
 'buy',
 'cake',
 'call',
 'car',
 'cat',
 'celebrating',
 'channel',
 'charge',
 'city',
 'class',
 'clean',
 'cleaned',
 'clothes',
 'code',
 'coffee',
 'cold',
 'colder',
 'color',
 'comedy',
 'coming',
 'commute',
 'computer',
 'connection',
 'cook',
 'cooking',
 'dance',
 'day',
 'days',
 'decided',
 'desk',
 'dies',
 'different',
 'dinner',
 'discussing',
 'do',
 'documentary',
 'doesnt',
 'draw',
 'dress',
 'drinks',
 'driving',
 'during',
 'earlier',
 'early',
 'eat',
 'eats',
 'email',
 'english',
 'enjoy',
 'enjoys',
 'eos',
 'evening',
 'evenings',
 'every',
 'everyone',
 'exam',
 'exams',
 'exercise'

In [ ]:
class Tokenizer:
  def __init__(self):
    super().__init__()
    self.word2idx = {}
    self.idx2word = {}
    self.word_count = 2

    self.word2idx["<PAD>"] = 0
    self.idx2word[0] = "<PAD>"
    self.word2idx["<UNK>"] = 1
    self.idx2word[1] = "<UNK>"

  def fit(self, corpus):
    words = []

    for line in corpus:
      line = line.lower()
      line = re.sub(r"[^a-zA-Z ]", "", line)
      for word in line.split():
        words.append(word)

    set_words = list(sorted(Counter(words)))

    for index, word in enumerate(set_words, start=2):
      self.word2idx[word] = index
      self.idx2word[index] = word
      self.word_count += 1

  def __len__(self):
    return self.word_count

  def encode(self, text):

    tokenized_words = []
    for word in text.split():
      if word in self.word2idx:
        tokenized_words.append(self.word2idx[word])
      else:
        tokenized_words.append(self.word2idx["<UNK>"])

    return tokenized_words

  def decode(self, idxs):
    decoded_words = []
    for idx in idxs:
      decoded_words.append(self.idx2word[idx])

    return decoded_words

In [ ]:
tokenizer = Tokenizer()

In [ ]:
tokenizer.fit(corpus)

In [ ]:
tokenizer.encode("<EOS>")

[1]

In [ ]:
tokenizer.decode([2])

['a']

In [ ]:
tokenizer.encode("I want to buy a car")

[1, 317, 292, 39, 2, 42]

In [ ]:
len(tokenizer)

339

In [ ]:
tokenizer.decode([140, 316, 291, 39, 3, 42])

['how', 'walking', 'time', 'buy', 'about', 'car']

# Prepare dataset

In [ ]:
input_sequences = []
output_sequence = []

In [ ]:
for line in corpus:
  line = line.lower()
  line = re.sub(r"[^a-zA-Z ]", "", line)

  token_list = tokenizer.encode(line)

  for _ in range(1, len(token_list)):
    n_gram = token_list[:_ + 1]

    input_sequences.append(n_gram[:-1])
    output_sequence.append(n_gram[-1])

In [ ]:
max_len_seq = max([len(seq) for seq in input_sequences])
max_len_seq

14

In [ ]:
padded_sequences = []

In [ ]:
for seq in input_sequences:
  pad_seq_with = max_len_seq - len(seq)
  padded_sequences.append([0] * pad_seq_with + seq)

In [ ]:
padded_sequences

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 141],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 141, 317],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 141, 317, 292],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 141, 317, 292, 39],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 141, 317, 292, 39, 2],
 [0, 0, 0, 0, 0, 0, 0, 0, 141, 317, 292, 39, 2, 197],
 [0, 0, 0, 0, 0, 0, 0, 141, 317, 292, 39, 2, 197, 60],
 [0, 0, 0, 0, 0, 0, 141, 317, 292, 39, 2, 197, 60, 38],
 [0, 0, 0, 0, 0, 141, 317, 292, 39, 2, 197, 60, 38, 141],
 [0, 0, 0, 0, 141, 317, 292, 39, 2, 197, 60, 38, 141, 73],
 [0, 0, 0, 141, 317, 292, 39, 2, 197, 60, 38, 141, 73, 200],
 [0, 0, 141, 317, 292, 39, 2, 197, 60, 38, 141, 73, 200, 128],
 [0, 141, 317, 292, 39, 2, 197, 60, 38, 141, 73, 200, 128, 185],
 [141, 317, 292, 39, 2, 197, 60, 38, 141, 73, 200, 128, 185, 89],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 254],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 254, 167],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 254, 167, 292],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 254, 167, 292, 241],
 [0, 0, 0, 0

In [ ]:
X = torch.tensor(padded_sequences, dtype=torch.long)
y = torch.tensor(output_sequence, dtype=torch.long)

# Create model

In [ ]:
class Generate(nn.Module):
  def __init__(self, vocab_size, embedding_dim, hidden_dim):
    super().__init__()
    self.vocab_size = vocab_size
    self.embedding_dim = embedding_dim
    self.hidden_dim = hidden_dim

    self.embedding = nn.Embedding(num_embeddings=vocab_size, embedding_dim=embedding_dim)
    self.lstm = nn.LSTM(embedding_dim, hidden_size=hidden_dim, batch_first=True)
    self.out = nn.Linear(hidden_dim, vocab_size)

  def forward(self, x):
    x = self.embedding(x)
    lstm_out, (_, _) = self.lstm(x)
    x = self.out(lstm_out[:, -1, :])

    return x

# model configuration

In [ ]:
model = Generate(len(tokenizer), embedding_dim=128, hidden_dim=32)

In [ ]:
optimizer = Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

In [ ]:
for epoch in range(200):
  model.train()

  outputs = model(X)
  optimizer.zero_grad()
  loss = criterion(outputs, y)

  correct = (outputs.argmax(dim = 1) == y).sum().item()
  loss.backward()

  optimizer.step()

  if epoch % 10 == 0:
    print(f"Epoch: {epoch + 1}, loss: {loss:.3f}, accuracy: {correct / y.size(0)}")


Epoch: 1, loss: 2.102, accuracy: 0.5788900979325353
Epoch: 11, loss: 1.994, accuracy: 0.6137105549510338
Epoch: 21, loss: 1.889, accuracy: 0.6332970620239391
Epoch: 31, loss: 1.789, accuracy: 0.661588683351469
Epoch: 41, loss: 1.695, accuracy: 0.6942328618063112
Epoch: 51, loss: 1.606, accuracy: 0.7279651795429815
Epoch: 61, loss: 1.522, accuracy: 0.7540805223068553
Epoch: 71, loss: 1.443, accuracy: 0.7758433079434167
Epoch: 81, loss: 1.369, accuracy: 0.7867247007616975
Epoch: 91, loss: 1.299, accuracy: 0.8063112078346029
Epoch: 101, loss: 1.234, accuracy: 0.8161044613710555
Epoch: 111, loss: 1.174, accuracy: 0.823721436343852
Epoch: 121, loss: 1.119, accuracy: 0.8302502720348205
Epoch: 131, loss: 1.068, accuracy: 0.8346028291621328
Epoch: 141, loss: 1.021, accuracy: 0.8422198041349293
Epoch: 151, loss: 0.978, accuracy: 0.8465723612622416
Epoch: 161, loss: 0.947, accuracy: 0.8443960826985855
Epoch: 171, loss: 0.904, accuracy: 0.8498367791077258
Epoch: 181, loss: 0.870, accuracy: 0.8520

In [ ]:
def generate_text(model, text, next_size, temperature):
  max_seq_len = 12
  model.eval()

  for _ in range(next_size):
    token_list = tokenizer.encode(text)

    pad_len = max_seq_len - len(token_list)
    if pad_len > 0:
      token_list = [0] * (pad_len) + token_list
    token_list = token_list[-max_seq_len:]
    inputs = torch.tensor(token_list, dtype=torch.long)

    with torch.no_grad():
      out = model(inputs.unsqueeze(0))

    probs = out / temperature
    probs = torch.softmax(probs, dim=1)

    predicted_token = torch.multinomial(probs, 1)
    predicted_word = tokenizer.decode([int(predicted_token)])
    if predicted_word[0] == "eos":
      break

    text = text + " " + predicted_word[0]

  return text


In [ ]:
probs = torch.multinomial(torch.tensor([0.3, 0.2, 0.5]), 1, replacement=True)
probs

tensor([0])

In [ ]:
generate_text(model, "she wants to buy", next_size=10, temperature=0.1)

'she wants to buy a new pair of shoes'